**BluePrint:**

1.   Initialize the Network
    - Adding the Hidden Layers (2 Layers, 16 Neurons Each)
    - Initializing Random Weights and Biases
    - Activation Functions (Softmax)
2.   Backpropagation
    - Calculate the Cost
    - Caclulate Gradient Descent for each weight and bias
    - Apply Learning Rate
3.   Train the model
    - Batch Stochastic Gradient Descent with Multiple Epochs
    - Feed Batches of Input Data into Model
    - Forward propagation: Calculate Output
    - Apply Backpropagation function
    - Calculate average training error for each Epoch

# Step 1: Initialize the network

In [3]:
import numpy as np

#defining activation function
def sigmoid(num):
    """
    Takes number as input
    Outputs the sigmoid transformation of that number
    """
    return 1.0/(1.0+np.exp(-num))

#defining softmax function
def softmax(ndarray):
    """
    Takes ndarray as input
    Outputs softmax transformation of that ndarray
    """
    return np.exp(ndarray)/sum(np.exp(ndarray))

#initialize the network
def initialize_net(neuron_nums_list):
    """
    Takes list as input, where:
        first element is number of neurons in input layer,
        last element is the number of neurons in the output layer,
        and intermediate elements are the numbers of neurons in the hidden layers.
        
    Outputs initialized activations, weights, and biases for these layers:
        Weights and biases are all random (0 to 1)
        Activations are 0 for the first layer, and calculated for all subsequent layers based on weights and biases
    
    Activation function for each layer is a sigmoid
    Output layer is put through softmax to get predicted probabilities that sum to 1
    """
    layers = len(neuron_nums_list)
    
    #first initializing lists of zeros with same length as the number of layers
    activations = [0]*layers
    biases = [0]*layers
    weights = [0]*layers
    
    #initializing random weights and biases, and 0 starting activations
    for i in range(layers):
        activations[i] = np.zeros(neuron_nums_list[i])
        if i > 0:
            biases[i] = np.random.uniform(0,1,size=neuron_nums_list[i])
            weights[i] = np.random.uniform(0,1,size=(neuron_nums_list[i], neuron_nums_list[i-1]))
    
    
    #calculating the first round of activations
    for i in range(layers):
        #skipping first layer
        if i == 0:
            continue
        #setting the activation using a sigmoid transformation of the linear combination of weights and previous activations
        #(plus bias)
        for j in range(len(activations[i])):
            pre_sigmoid = sum(activations[i-1]*weights[i][j]) + biases[i][j]
            activations[i][j] = sigmoid(pre_sigmoid)
            
            
    #putting the output layer through softmax to get predicted probabilities
    activations[-1] = softmax(activations[-1])
        
    return activations, biases, weights

In [4]:
activations, biases, weights = initialize_net([2,3,4])

In [5]:
activations

[array([0., 0.]),
 array([0.52303616, 0.55127872, 0.5255474 ]),
 array([0.2457346 , 0.25197217, 0.25053486, 0.25175838])]

In [6]:
biases

[0,
 array([0.09220993, 0.20583858, 0.10227866]),
 array([0.09190455, 0.79140614, 0.39507788, 0.35843415])]

In [7]:
weights

[0, array([[0.95170116, 0.6346612 ],
        [0.90073071, 0.08812915],
        [0.82545912, 0.41721607]]), array([[0.77814931, 0.59782574, 0.61863359],
        [0.43827772, 0.07625345, 0.44477637],
        [0.76134475, 0.70801143, 0.15075586],
        [0.49487974, 0.61003293, 0.64281701]])]

In [8]:
sum(activations[-1])

1.0

# Step 2: Backpropogation

In [15]:
#setting up example target, where the first category is correct
target = [1,0,0,0]

def cat_cross_entropy(actual, predicted):
    """
    cost function that calculates categorical cross-entropy for a single observation
    takes in arrays representing one-hot encoded vectors of actual and predicted status
    returns categorical cross-entropy (numeric)
    """
    return -sum(actual*np.log(predicted))



In [16]:
cat_cross_entropy(np.array([1,0,0,0]), np.array([.90,.08,.01,.01]))

0.10536051565782628

In [17]:
cat_cross_entropy(np.array([1,0,0,0]), np.array([.40,.3,.15,.15]))

0.916290731874155

# Neural Network Class from textbook Neural Networks and Deep Learning
Credit to Michael Nielsen
Found at http://neuralnetworksanddeeplearning.com/

With a few changes:
    1. Changing cost function to categorical cross-entropy
    2. Changing output activations to a softmax function, rather than argmax
    3. Slight modification to port from Python 2 to Python 3
    
Resources for doing this were found at https://deepnotes.io/softmax-crossentropy#the-softmax-function


In [2]:
#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):
    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print('Epoch {0}: {1} / {2}'.format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y)*sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))



############################################
## Added softmax function
def softmax(X):
    exps = np.exp(X-np.max(X))
    return exps / np.sum(exps)


## Added categorical cross-entropy function
def cat_cross_entropy(actual, predicted):
    """
    cost function that calculates categorical cross-entropy for a single observation
    takes in arrays representing one-hot encoded vectors of actual and predicted status
    returns categorical cross-entropy (numeric)
    """
    return -np.sum(actual*np.log(predicted))



## Added cross-entropy function
def cross_entropy(X,y):
    """
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    	Note that y is not one-hot encoded vector. 
    	It can be computed as y.argmax(axis=1) from one-hot encoded vectors of labels if required.
    """
    m = y.shape[0]
    p = softmax(X)
    # We use multidimensional array indexing to extract 
    # softmax probability of the correct label for each sample.
    # Refer to https://docs.scipy.org/doc/numpy/user/basics.indexing.html#indexing-multi-dimensional-arrays for understanding multidimensional array indexing.
    log_likelihood = -np.log(p[range(m),y])
    loss = np.sum(log_likelihood) / m
    return loss

def delta_cross_entropy(X,y):
    """
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    	Note that y is not one-hot encoded vector. 
    	It can be computed as y.argmax(axis=1) from one-hot encoded vectors of labels if required.
    """
    m = y.shape[0]
    grad = softmax(X)
    grad[range(m),y] -= 1
    grad = grad/m
    return grad

# Code for loading MNIST data

Also from Michael Nielsen: https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/mnist_loader.py

With slight modifications to port from python2 to python3

In [3]:
#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [4]:
training_data, validation_data, test_data = load_data_wrapper()

In [5]:
net = Network([784, 30, 10])

In [6]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9040 / 10000
Epoch 1: 9213 / 10000
Epoch 2: 9337 / 10000
Epoch 3: 9369 / 10000
Epoch 4: 9404 / 10000
Epoch 5: 9422 / 10000
Epoch 6: 9399 / 10000
Epoch 7: 9451 / 10000
Epoch 8: 9435 / 10000
Epoch 9: 9432 / 10000
Epoch 10: 9462 / 10000
Epoch 11: 9443 / 10000
Epoch 12: 9457 / 10000
Epoch 13: 9446 / 10000
Epoch 14: 9464 / 10000
Epoch 15: 9470 / 10000
Epoch 16: 9477 / 10000
Epoch 17: 9479 / 10000
Epoch 18: 9465 / 10000
Epoch 19: 9494 / 10000
Epoch 20: 9492 / 10000
Epoch 21: 9503 / 10000
Epoch 22: 9478 / 10000
Epoch 23: 9502 / 10000
Epoch 24: 9492 / 10000
Epoch 25: 9511 / 10000
Epoch 26: 9515 / 10000
Epoch 27: 9495 / 10000
Epoch 28: 9474 / 10000
Epoch 29: 9488 / 10000


In [12]:
x, y = zip(*training_data)

ValueError: not enough values to unpack (expected 2, got 0)

[]

AttributeError: 'list' object has no attribute 'shape'

[]